In [1]:
import os
import glob
import pandas as pd
import geopandas as gpd

In [2]:
"""
Importante: Os diretórios, in e out são baseados no mapeamento da máquina onde estão os dados!
Caso queira adaptar, é importante verificar a possibilidade de mudar essas variáveis e o código abaixo
de mudança de pasta. 
"""
os.chdir('../..') #Retorna duas bastas.

In [3]:
path_in = os.path.join(os.getcwd(),r'DADOS\SHP\PROCESSAMENTOS_MAPBIOMAS - PROFLORA CORRIGIDO\7_METRICAS SIMPLES - PROFLORA CORRIGIDO')

In [4]:
#Chegar se a pasta PROFLORA_CORRIGIDO existe ou não. Caso contrário, cria a pasta.
if not os.path.exists(r'DADOS\TAB\PROFLORA_CORRIGIDO'):
    os.makedirs(r'DADOS\TAB\PROFLORA_CORRIGIDO')

path_out = os.path.join(os.getcwd(),r'DADOS\TAB\PROFLORA_CORRIGIDO')

In [5]:
files = glob.glob(path_in + '\*.shp')

In [6]:
gdf1 = gpd.read_file(files[0])

In [7]:
def legenda(df):

    if df['DN'] == 1:
        return '1 - Formação Florestal'
    elif df['DN'] == 2:
        return '2 - Formação Savânica'
    elif df['DN'] == 3:
        return '3 - Campo Alagado e Área Pantanosa'
    elif df['DN'] == 4:
        return '4 - Formação Campestre'
    elif df['DN'] == 5:
        return '5 - Afloramento Rochoso'
    elif df['DN'] == 6:
       return '6 - Pastagem'
    elif df['DN'] == 7:
        return '7 - Agropecuária'
    elif df['DN'] == 8:
        return ' 8 - Silvicultura'
    elif df['DN'] == 9:
        return '9 - Mosaico de Usos'
    elif df['DN'] == 10:
        return '10 - Área Urbanizada'
    elif df['DN'] == 11:
       return '11 - Mineração'
    elif df['DN'] == 12:
        return '12 - Outras Áreas não Vegetadas'
    else:
        return '13 - Rio, Lago e Oceano'

In [8]:
for file in files:
    #Pegar o nome do arquivo
    nm_arquivo = os.path.basename(file).split('.')[0]

    #Ler o shp e transformar em um geodf
    gdf = gpd.read_file(file)
    
    #Calcular área m2 e perímetro
    gdf['area_m2'] = gdf.area
    gdf['perimetro'] = gdf.length

    gdf['legenda'] = gdf.apply(legenda, axis=1)
    #Salvar o arquivo shp com as métricas calculadas
    gdf.to_file(
        os.path.join(
            os.getcwd(),r'DADOS\SHP\PROCESSAMENTOS_MAPBIOMAS - PROFLORA CORRIGIDO\6_METRICAS SIMPLES') + f'\{nm_arquivo}.shp'
    )

    #Transformar em um dataframe e dropar a coluna geometry
    df = pd.DataFrame(gdf.drop(columns='geometry'))

    #Agrupar as colunas DN, nome e legenda e resetar o index
    df2 = df.groupby(['DN','nome','legenda']).sum(['area_m2','perimetro']).reset_index()

    #Ordenar pela Área de Proteção Ambiental
    df2 = df2.sort_values(by=['nome','legenda'])

    #Criar a coluna Classificação
    df2['Classificação'] = df2['legenda'].astype(str)

    #Ordenar as colunas
    df2 = df2[[ 
                'nome',
                'Classificação',
                'area_m2',
                'perimetro'
            ]]
    
    #Renomear as colunas
    df2 = df2.rename({
                        'nome':'Nome',
                        'area_m2':'Área (m²)',
                        'perimetro':'Perímetro'
                },
                axis='columns'
                )

    #Salvar em csv
    df2.to_csv(path_out + f'\{nm_arquivo}.csv',
               encoding='latin',
               index=False)
    